# ORIE 5258 / Homework 5: Random Forests

#### Students: Vishakha Korde (vk369) and Luis Alonso Cendra Villalobos (lc2234)

**Stocks chosen:** AAPL, AMZN, BRK, GOOG, META, MSFT, NVDA, TSLA, UNH, XOM

**Stock to be predicted:** AAPL

**Time series:**

Start = 1/1/2018

End = 11/04/2023

In [150]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [151]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import shift


In [152]:
# creating pandas dataframe for each stock

path = '/content/drive/MyDrive/ORIE5258 - Python for Finance/HW3/'

AAPL = pd.read_csv(path + 'AAPL.csv')
AAPL.Name = 'AAPL'
AMZN = pd.read_csv(path + 'AMZN.csv')
AMZN.Name = 'AMZN'
BRK  = pd.read_csv(path + 'BRK.csv')
BRK.Name = 'BRK'
GOOG = pd.read_csv(path + 'GOOG.csv')
GOOG.Name = 'GOOG'
META = pd.read_csv(path + 'META.csv')
META.Name = 'META'
MSFT = pd.read_csv(path + 'MSFT.csv')
MSFT.Name = 'MSFT'
NVDA = pd.read_csv(path + 'NVDA.csv')
NVDA.Name = 'NVDA'
TSLA = pd.read_csv(path + 'TSLA.csv')
TSLA.Name = 'TSLA'
UNH  = pd.read_csv(path + 'UNH.csv')
UNH.Name = 'UNH'
XOM  = pd.read_csv(path + 'XOM.csv')
XOM.Name = 'XOM'


In [153]:
# List of stocks
dataframes = [AAPL, AMZN, BRK, GOOG, META, MSFT, NVDA, TSLA, UNH, XOM]

# Filter Adj Cls for each stock
for df in dataframes:
    df[df.Name] = df['Adj Close']

In [154]:
# Merge DataFrames for all asset prices
for df in dataframes:
  asset_prices = pd.concat([AAPL['Date'], AAPL['AAPL'], AMZN['AMZN'],
                            BRK['BRK'], GOOG['GOOG'], META['META'], MSFT['MSFT'],
                            NVDA['NVDA'], TSLA['TSLA'], UNH['UNH'],
                            XOM['XOM']], axis=1)

# Calculate returns data for 1 day window

# Computing % returns for each stock
for df in dataframes:
    df[df.Name] = df['Adj Close'].shift(-1) / df['Adj Close'] - 1


# Merge DataFrames for all asset returns
for df in dataframes:
  asset_returns = pd.concat([AAPL['Date'], AAPL['AAPL'], AMZN['AMZN'],
                             BRK['BRK'], GOOG['GOOG'], META['META'], MSFT['MSFT'],
                             NVDA['NVDA'], TSLA['TSLA'], UNH['UNH'],
                             XOM['XOM']], axis=1)

# Fill NaN values with zeros
asset_returns = asset_returns.fillna(0)

#Remove the last line because its filled with zeros
asset_returns = asset_returns.iloc[:-1]

# Print the asset returns
print(asset_returns.iloc[:, :])


            Date      AAPL      AMZN       BRK      GOOG      META      MSFT  \
0     2018-01-02 -0.000174  0.012775  0.013031  0.016413  0.017914  0.004654   
1     2018-01-03  0.004645  0.004476  0.004505  0.003621 -0.001841  0.008802   
2     2018-01-04  0.011386  0.016163  0.003637  0.014571  0.013671  0.012398   
3     2018-01-05 -0.003714  0.014425  0.006554  0.004273  0.007653  0.001020   
4     2018-01-08 -0.000115  0.004676  0.001381 -0.000614 -0.002178 -0.000680   
...          ...       ...       ...       ...       ...       ...       ...   
1465  2023-10-27  0.012305  0.038907  0.017184  0.019044  0.019984  0.022740   
1466  2023-10-30  0.002819  0.002863  0.011618 -0.003579 -0.004593  0.002372   
1467  2023-10-31  0.018739  0.029379  0.007090  0.018116  0.035118  0.023543   
1468  2023-11-01  0.020693  0.007810  0.015331  0.007917 -0.003143  0.006502   
1469  2023-11-02 -0.005181  0.003839  0.007994  0.013921  0.011999  0.012862   

          NVDA      TSLA       UNH     

In [155]:
# Definition of classes according to sgn(AAPL_returns)

asset_returns.loc[asset_returns['AAPL'] < 0, 'y'] = 0 # negative class
asset_returns.loc[(asset_returns['AAPL'] >= 0), 'y'] = 1 # non-negative class

#Drop the AAPL column
asset_returns = asset_returns.drop(['AAPL'], axis=1)

print(asset_returns)

            Date      AMZN       BRK      GOOG      META      MSFT      NVDA  \
0     2018-01-02  0.012775  0.013031  0.016413  0.017914  0.004654  0.065814   
1     2018-01-03  0.004476  0.004505  0.003621 -0.001841  0.008802  0.005271   
2     2018-01-04  0.016163  0.003637  0.014571  0.013671  0.012398  0.008474   
3     2018-01-05  0.014425  0.006554  0.004273  0.007653  0.001020  0.030641   
4     2018-01-08  0.004676  0.001381 -0.000614 -0.002178 -0.000680 -0.000270   
...          ...       ...       ...       ...       ...       ...       ...   
1465  2023-10-27  0.038907  0.017184  0.019044  0.019984  0.022740  0.016321   
1466  2023-10-30  0.002863  0.011618 -0.003579 -0.004593  0.002372 -0.009256   
1467  2023-10-31  0.029379  0.007090  0.018116  0.035118  0.023543  0.037886   
1468  2023-11-01  0.007810  0.015331  0.007917 -0.003143  0.006502  0.027903   
1469  2023-11-02  0.003839  0.007994  0.013921  0.011999  0.012862  0.034455   

          TSLA       UNH       XOM    y

In [156]:
# Splitting - Testing data

from sklearn.model_selection import train_test_split

X = asset_returns.drop(['y', 'Date'], axis=1)
y = asset_returns['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

## 1) Bagging

In [157]:
# Bagging

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestClassifier(max_depth=2, random_state=0)
forest.fit(X_train, y_train)

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train)

print(rf)

RandomForestRegressor(n_estimators=1000, random_state=42)


In [158]:
# Measure model performance

from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = forest.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.760
cm =  [[138  67]
 [ 39 197]]
cm[0,0] =  138


## 2) Boosting

In [159]:
# sklearn.ensemble.GradientBoostingClassifier¶
from xgboost import XGBRFClassifier

gbc = XGBRFClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# Measure model performance
from sklearn.metrics import accuracy_score
y_pred = gbc.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.760
cm =  [[153  52]
 [ 54 182]]
cm[0,0] =  153


## 3) Stacking

In [160]:

from sklearn.ensemble import StackingClassifier
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
 ('rf100', RandomForestClassifier(n_estimators=100, random_state=42)),
 ('rf20', RandomForestClassifier(n_estimators=20, random_state=42))]
model = StackingClassifier(estimators=estimators)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.762
cm =  [[144  61]
 [ 44 192]]
cm[0,0] =  144


# Appendix

## 1) Additional work on Bagging

In [140]:
# 1) ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(n_estimators=100, random_state=0)
extra.fit(X_train, y_train)
extra.predict(X_test)

array([1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0.

In [141]:
# Measure model performance

from sklearn.metrics import accuracy_score
y_pred = extra.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.778
cm =  [[146  59]
 [ 39 197]]
cm[0,0] =  146


In [142]:
# 2) BaggingClassifier

from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(estimator=SVC(), n_estimators=10, random_state=0).fit(X_train, y_train)

In [143]:
# Measure model performance

from sklearn.metrics import accuracy_score
y_pred = bag.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.766
cm =  [[145  60]
 [ 43 193]]
cm[0,0] =  145


## 2) Additional work on Boosting

In [144]:

# 1) GradientBoostingClassifier¶
from sklearn.ensemble import GradientBoostingClassifier

gbooc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# Measure model performance
from sklearn.metrics import accuracy_score
y_pred = gbooc.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.771
cm =  [[143  62]
 [ 39 197]]
cm[0,0] =  143


In [145]:
# 2) HistGradientBoostingClassifier

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2

hist_gbooc = HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)

In [146]:
# Measure model performance
from sklearn.metrics import accuracy_score
y_pred = hist_gbooc.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.773
cm =  [[145  60]
 [ 40 196]]
cm[0,0] =  145


## 3) Additional work on Stacking

In [147]:
# 3) Stacking
# sklearn.ensemble.StackingClassifier

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define base classifiers
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
gb_classifier = GradientBoostingClassifier(n_estimators=50, random_state=42)
svc_classifier = SVC(probability=True, random_state=42)

# Define the meta-classifier
meta_classifier = LogisticRegression(random_state=42)

# Create the stacking classifier
stacking_classifier = StackingClassifier(
    estimators=[('rf', rf_classifier), ('gb', gb_classifier), ('svc', svc_classifier)],
    final_estimator=meta_classifier,
    stack_method='auto',  # You can also specify the method for feature stacking
    cv=5  # Number of cross-validation folds
)

# Fit the stacking classifier on the training data
stacking_classifier.fit(X_train, y_train)



StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(n_estimators=50,
                                                       random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(n_estimators=50,
                                                           random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=LogisticRegression(random_state=42))

In [148]:
# Measure model performance
from sklearn.metrics import accuracy_score
y_pred = stacking_classifier.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('cm = ', cm)
print('cm[0,0] = ', cm[0,0])

Accuracy: 0.764
cm =  [[145  60]
 [ 44 192]]
cm[0,0] =  145
